In [17]:
import sys
import os
from collections import namedtuple

In [18]:

Entry = namedtuple('Entry',['ts','cluster'])
Writer = namedtuple('Writer',['start','end','fid'])

In [19]:
def makeCounters(fPrefix,start,length,overlap,num):
    c = list()
    for x in range(num):
        s = float(start) + length*x
        e =  s + length
        s0 = s + length/2
        e0 = s0 + length
        
        w1 = Writer(s,e,open('%s%i-%i.out' % (fPrefix,int(s),int(e)),'w'))
        w2 = Writer(s0,e0,open('%s%i-%i.out' % (fPrefix,int(s0),int(e0)),'w'))
        
        c.append(w1)
        c.append(w2)
        
    return c
      
    

In [20]:
length=5*60*60
overlap=2.5*60*60

In [21]:
def putLogLine(entry,writers):
    putSomewhere = False
    for w in writers:
        if (float(entry.ts) >= float(w.start)) and (float(entry.ts) <= float(w.end)):
            w.fid.write('%s ' % (entry.cluster))
            putSomewhere = True
        else:
            if float(entry.ts) > float(w.end):
                if not w.fid.closed:
                    w.fid.write('\n')
                else:
                    w.fid.close()
                
    if not putSomewhere:
        print 'BAD BAD BAD BAD'
        
    for w in writers:
        if not w.fid.closed:
            return False
    return True

In [22]:
def writeThings(f,fPrefix,length,overlap):
    initialized = False
    startingPoint = 0.0
    writers = list()
    read = 0
    for current in f.readlines():
        
        c,d,e = current.split(',',2)
        entry = Entry(c,d)
        if not initialized:
            startingPoint = entry.ts
            initialized = True
            writers = makeCounters(fPrefix,startingPoint,length,overlap,10)
        moreNeeded = putLogLine(entry,writers)
        read += 1
        if moreNeeded:
            print 'moar writers'
            startingPoint = entry.ts
            writers = makeCounters(fPrefix,startingPoint,length,overlap,10)
            putLogLine(entry,writers)
    
    for w in writers:
        w.fid.close()
    
    print 'read:',read

In [23]:
os.chdir('/Users/dgrossman/data')

In [25]:
writeThings(open('./tbird.500k.normal.350Support.out.clustered.noJunk','r'),'tbirdOverlap',length,overlap)

read: 326094


In [24]:
pwd()

u'/Users/dgrossman/data'